In [12]:
# sync python module
%load_ext autoreload
%autoreload 2

## Gen pretrain data dict json

In [44]:
workspace_dir = '/nfs/Workspace/CardiacSeg'
import sys

# set package path
sys.path.append(workspace_dir)

from datasets.chgh_dataset import get_data_dicts
from data_utils.io import save_json
from expers.preprocess import get_rel_data_dicts

exp_name = 'exp_2_2'
root_data_dir = os.path.join(workspace_dir, 'dataset', 'chgh')
data_dir = os.path.join(root_data_dir, 'dataset_2')
data_dicts_json = os.path.join(root_data_dir, 'data_dicts', f'{exp_name}_pretrain.json')

data_dicts = get_rel_data_dicts(get_data_dicts(data_dir))
data_dicts.pop(11)
data_dicts.pop(-3)
data_dicts[-5:-2]

# data_dicts

tr_data_dicts = [*data_dicts[:-5], *data_dicts[-2:]]
val_data_dicts = data_dicts[-5:-2]
tt_data_dicts = data_dicts[-5:-2]

out_data_dict = {
    'train':tr_data_dicts,
    'val':val_data_dicts,
    'test':tt_data_dicts
}

save_json(out_data_dict, data_dicts_json)
out_data_dict

save json to /nfs/Workspace/CardiacSeg/dataset/chgh/data_dicts/exp_2_2_pretrain.json


{'train': [{'image': 'pid_05/pid_05.nii.gz',
   'label': 'pid_05/pid_05_gt.nii.gz'},
  {'image': 'pid_06/pid_06.nii.gz', 'label': 'pid_06/pid_06_gt.nii.gz'},
  {'image': 'pid_07/pid_07.nii.gz', 'label': 'pid_07/pid_07_gt.nii.gz'},
  {'image': 'pid_08/pid_08.nii.gz', 'label': 'pid_08/pid_08_gt.nii.gz'},
  {'image': 'pid_21/pid_21.nii.gz', 'label': 'pid_21/pid_21_gt.nii.gz'},
  {'image': 'pid_23/pid_23.nii.gz', 'label': 'pid_23/pid_23_gt.nii.gz'},
  {'image': 'pid_24/pid_24.nii.gz', 'label': 'pid_24/pid_24_gt.nii.gz'},
  {'image': 'pid_27/pid_27.nii.gz', 'label': 'pid_27/pid_27_gt.nii.gz'},
  {'image': 'pid_30/pid_30.nii.gz', 'label': 'pid_30/pid_30_gt.nii.gz'},
  {'image': 'pid_34/pid_34.nii.gz', 'label': 'pid_34/pid_34_gt.nii.gz'},
  {'image': 'pid_57/pid_57.nii.gz', 'label': 'pid_57/pid_57_gt.nii.gz'},
  {'image': 'pid_1002/pid_1002.nii.gz',
   'label': 'pid_1002/pid_1002_gt.nii.gz'},
  {'image': 'pid_1003/pid_1003.nii.gz',
   'label': 'pid_1003/pid_1003_gt.nii.gz'}],
 'val': [{'image

## Pretrain

In [51]:
workspace_dir = '/nfs/Workspace/CardiacSeg'

exp_name = 'exp_1'
model_name = 'unetcnx_x3_2_2'
data_name = 'chgh'

root_data_dir = os.path.join(workspace_dir, 'dataset', data_name)
data_dir = os.path.join(root_data_dir, 'dataset_2')
data_dicts_json = os.path.join(root_data_dir, 'data_dicts', f'exp_2_2_pretrain.json')

root_dir = os.path.join(
    workspace_dir,
    'exps',
    'exps',
    model_name,
    data_name,
    'pretrain',
    exp_name
)
model_dir = os.path.join(root_dir, 'models', 'pretrain', exp_name)
log_dir = os.path.join(root_dir, 'logs', 'pretrain', exp_name)

os.makedirs(root_dir, exist_ok=True)

best_model_path = os.path.join(model_dir, 'model_bestValRMSE.pt')
final_model_path = os.path.join(model_dir, 'model_final.pt')

%cd {root_dir}

/nfs/Workspace/CardiacSeg/exps/exps/unetcnx_x3_2_2/chgh/pretrain/exp_1


In [58]:
!PYTHONPATH={workspace_dir} /opt/conda/bin/python {workspace_dir}/expers/pretrain/train.py\
--use_checkpoint \
--batch_size=1 \
--num_steps=100000 \
--lrdecay \
--eval_num=100 \
--max_early_stop_count=60 \
--model_name={model_name} \
--logdir={log_dir} \
--model_dir={model_dir} \
--data_dirs={data_dir} \
--data_dicts_jsons={data_dicts_json} \
--resume={final_model_path}
# --lr=<Lr> 
# --roi_x=<Roi_x> \
# --roi_y=<Roi_y> \
# --roi_z=<Roi_z> \

/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
cuda is available
Training with a single process on 1 GPUs.
model: unetcnx_x3_2_2
use warmup cosine schedule
load data dict jsons: ['/nfs/Workspace/CardiacSeg/dataset/chgh/data_dicts/exp_2_2_pretrain.json']
load json from /nfs/Workspace/CardiacSeg/dataset/chgh/data_dicts/exp_2_2_pretrain.json
train files (13): ['pid_05', 'pid_06', 'pid_07', 'pid_08', 'pid_21', 'pid_23', 'pid_24', 'pid_27', 'pid_30', 'pid_34', 'pid_57', 'pid_1002', 'pid_1003']
val files (3): ['pid_107', 'pid_108', 'pid_110']
test files (3): ['pid_107', 'pid_108', 'pid_110']
Dataset 1 ['/nfs/Workspace/Cardi